In [57]:
# Importing libraries
import os
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime, timedelta
import math
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import yfinance as yf

from neurotrader.download_data import download_df

# Reversal Pattern Detection Algorithm
- Look at the 1 day intervals for now
- Want to look at min by min at some point
 

In [58]:
# Trendline parameter
lookback = 30

data = download_df('AAPL', '1y', '1d')
data.head()

[*********************100%***********************]  1 of 1 completed


KeyError: 'Adj_Close'

In [24]:
arr = data['close'].to_numpy()
bottoms = scipy.signal.argrelextrema(arr, np.less, order=3)
tops = scipy.signal.argrelextrema(arr, np.greater, order=3)

In [25]:
bottoms

(array([  2,  12,  18,  31,  41,  50,  63,  71,  90, 108, 118, 139, 162,
        187, 194, 204, 217, 229, 242], dtype=int64),)

In [26]:
tops

(array([  7,  14,  19,  23,  29,  38,  47,  57,  69,  77,  87,  94, 102,
        112, 116, 129, 156, 173, 190, 198, 209, 219, 232, 238, 243, 251],
       dtype=int64),)